## Energy saved from recycling
<p>Did you know that recycling saves energy by reducing or eliminating the need to make materials from scratch? For example, aluminum can manufacturers can skip the energy-costly process of producing aluminum from ore by cleaning and melting recycled cans. Aluminum is classified as a non-ferrous metal.</p>
<p>Singapore has an ambitious goal of becoming a zero-waste nation. The amount of waste disposed of in Singapore has increased seven-fold over the last 40 years. At this rate, Semakau Landfill, Singapore’s only landfill, will run out of space by 2035. Making matters worse, Singapore has limited land for building new incineration plants or landfills.</p>
<p>The government would like to motivate citizens by sharing the total energy that the combined recycling efforts have saved every year. They have asked you to help them.</p>
<p>You have been provided with three datasets. The data come from different teams, so the names of waste types may differ.</p>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:16px"><b>datasets/wastestats.csv - Recycling statistics per waste type for the period 2003 to 2017</b>
    </div>
    <div>Source: <a href="https://www.nea.gov.sg/our-services/waste-management/waste-statistics-and-overall-recycling">Singapore National Environment Agency</a></div>
<ul>
    <li><b>waste_type: </b>The type of waste recycled.</li>
    <li><b>waste_disposed_of_tonne: </b>The amount of waste that could not be recycled (in metric tonnes).</li>
    <li><b>total_waste_recycle_tonne: </b>The amount of waste that could be recycled (in metric tonnes).</li>
    <li><b>total_waste_generated: </b>The total amount of waste collected before recycling (in metric tonnes).</li>
    <li><b>recycling_rate: </b>The amount of waste recycled per tonne of waste generated.</li>
    <li><b>year: </b>The recycling year.</li>
</ul>
    </div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">
    <div style="font-size:16px"><b>datasets/2018_2019_waste.csv - Recycling statistics per waste type for the period 2018 to 2019</b>
    </div>
    <div> Source: <a href="https://www.nea.gov.sg/our-services/waste-management/waste-statistics-and-overall-recycling">Singapore National Environment Agency</a></div>
<ul>
    <li><b>Waste Type: </b>The type of waste recycled.</li>
    <li><b>Total Generated: </b>The total amount of waste collected before recycling (in thousands of metric tonnes).</li> 
    <li><b>Total Recycled: </b>The amount of waste that could be recycled. (in thousands of metric tonnes).</li>
    <li><b>Year: </b>The recycling year.</li>
</ul>
    </div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">
    <div style="font-size:16px"><b>datasets/energy_saved.csv -  Estimations of the amount of energy saved per waste type in kWh</b>
    </div>
<ul>
    <li><b>material: </b>The type of waste recycled.</li>
    <li><b>energy_saved: </b>An estimate of the energy saved (in kiloWatt hour) by recycling a metric tonne of waste.</li> 
    <li><b>crude_oil_saved: </b>An estimate of the number of barrels of oil saved by recycling a metric tonne of waste.</li>
</ul>

</div>
<pre><code>
</code></pre>

In [75]:
# Import necessary libraries
import numpy as np
import pandas as pd

In [81]:
# get 2015 to 2017 data
waste1 = pd.read_csv('datasets/wastestats.csv', dtype={'total_waste_recycled_tonne':'int', 'waste_type':'str'})

# Getting rid of useless columns
waste1.drop(["waste_disposed_of_tonne", "total_waste_generated_tonne", "recycling_rate"], inplace=True, axis=1)

# Renaming columns for merging with the other dataset
waste1.columns = ['waste_type', 'waste_recycled', 'year']

# Filtering out unneeded years and waste types
waste1.query('year >= 2015', inplace=True)
recycled = ['Ferrous metal', 'Glass', 'Plastics', 'Plastic', 'Non-ferrous metal', 'Non-ferrous metals']
waste1= waste1[waste1.waste_type.isin(recycled)]

# Cleaning the waste_type column
waste1['waste_type'].replace('Plastic','Plastics',inplace=True)
waste1['waste_type'].replace(['Non-ferrous metals','Non-ferrous metal'],'Non-Ferrous Metal',inplace=True)
waste1['waste_type'].replace('Ferrous metal', 'Ferrous Metal',inplace=True)

# Pivoting the dataset by year
waste1_pivoted = waste1.pivot_table(index='year', columns='waste_type', values='waste_recycled', aggfunc='sum')
waste1_pivoted



waste_type,Ferrous Metal,Glass,Non-Ferrous Metal,Plastics
year,,,,
2015,1333300,14600,160400,57800
2016,1351500,14700,95900,59500
2017,1371000,12400,92200,51800


In [83]:
# get 2018 to 2019
waste2 = pd.read_csv('datasets/2018_2019_waste.csv')

# Filtering columns and renaming
waste2.drop("Total Generated ('000 tonnes)", inplace=True, axis=1)
waste2.columns = ['waste_type', 'waste_recycled', 'year']

# Filtering out unneeded waste types
recycled2 = ['Plastics', 'Non-Ferrous Metal', 'Glass', 'Ferrous Metal']
waste2= waste2[waste2['waste_type'].isin(recycled2)]

# Converting to metric tonne
waste2.waste_recycled = waste2.waste_recycled * 1000

# Pivoting the dataset by year
waste2_pivoted=waste2.pivot_table(index='year', columns='waste_type', values='waste_recycled', aggfunc='sum')

# Merging the two datasets
waste = pd.concat([waste1_pivoted, waste2_pivoted])
display(waste)

waste_type,Ferrous Metal,Glass,Non-Ferrous Metal,Plastics
year,,,,
2015,1333300,14600,160400,57800
2016,1351500,14700,95900,59500
2017,1371000,12400,92200,51800
2018,126000,12000,170000,41000
2019,1270000,11000,124000,37000


In [84]:
# Getting the energy data with quick filtering
energy = pd.read_csv('datasets/energy_saved.csv', header=3)
energy.drop(["material", "Paper"], inplace=True, axis=1)
display(energy)

# Calculating the kiloWatt hour for the saved energy
waste['Ferrous Metal'] = waste['Ferrous Metal'] * 642
waste['Glass'] = waste['Glass'] * 42
waste['Plastics'] = waste['Plastics'] * 5774
waste['Non-Ferrous Metal'] = waste['Non-Ferrous Metal'] * 14000

# Getting the sum of saved energy across relevant materials
waste['total_energy_saved'] = waste.sum(axis=1)
display(waste)

,Plastic,Glass,Ferrous Metal,Non-Ferrous Metal
0,5774 Kwh,42 Kwh,642 Kwh,14000 Kwh
1,16 barrels,NaN,1.8 barrels,40 barrels


waste_type,Ferrous Metal,Glass,Non-Ferrous Metal,Plastics,total_energy_saved
year,,,,,
2015,855978600,613200,2245600000,333737200,3435929000
2016,867663000,617400,1342600000,343553000,2554433400
2017,880182000,520800,1290800000,299093200,2470596000
2018,80892000,504000,2380000000,236734000,2698130000
2019,815340000,462000,1736000000,213638000,2765440000


In [79]:
# The desired output
annual_energy_savings = pd.DataFrame(waste, columns=['total_energy_saved'])
display(annual_energy_savings)

,total_energy_saved
year,
2015,3435929000
2016,2554433400
2017,2470596000
2018,2698130000
2019,2765440000
